In [1]:
import numpy, pandas, sys, cv2, os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD

C:\Programs\Anaconda3\envs\ai\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
neu_data_dir = 'neu-data'
train_dir = os.path.join(neu_data_dir, 'train')
val_dir = os.path.join(neu_data_dir, 'val')
test_dir = os.path.join(neu_data_dir, 'test')
img_width, img_height = 200, 200
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 100
# Размер мини-выборки
batch_size = 16
# Количество изображений для обучения
nb_train_samples = 1254
# Количество изображений для проверки
nb_validation_samples = 270
# Количество изображений для тестирования
nb_test_samples = 276
classes = 6

In [3]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [5]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1254 images belonging to 6 classes.


In [6]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 270 images belonging to 6 classes.


In [7]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 276 images belonging to 6 classes.


In [8]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=50,
        validation_data=val_generator,
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/50
78/78 [==============================] - 11s 141ms/step - loss: 0.5224 - acc: 0.8177 - val_loss: 0.4671 - val_acc: 0.8333
Epoch 2/50
78/78 [==============================] - 10s 125ms/step - loss: 0.4635 - acc: 0.8255 - val_loss: 0.4204 - val_acc: 0.8340
Epoch 3/50
78/78 [==============================] - 10s 124ms/step - loss: 0.4169 - acc: 0.8318 - val_loss: 0.4066 - val_acc: 0.8346
Epoch 4/50
78/78 [==============================] - 10s 124ms/step - loss: 0.3893 - acc: 0.8387 - val_loss: 0.3709 - val_acc: 0.8639
Epoch 5/50
78/78 [==============================] - 10s 129ms/step - loss: 0.3733 - acc: 0.8408 - val_loss: 0.4004 - val_acc: 0.8581
Epoch 6/50
78/78 [==============================] - 10s 124ms/step - loss: 0.3454 - acc: 0.8502 - val_loss: 0.3824 - val_acc: 0.8503
Epoch 7/50
78/78 [==============================] - 10s 123ms/step - loss: 0.3371 - acc: 0.8543 - val_loss: 0.3488 - val_acc: 0.8776
Epoch 8/50
78/78 [==============================] - 10s 122ms/step - 

In [9]:
import h5py

In [10]:
model.save_weights("lenet2.h5")


In [11]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [12]:
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

Точность на тестовых данных: 95.96%
